In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print(os.walk('/kaggle/working'))

In [ ]:
import io
import os
import bson
from tqdm import tqdm
import matplotlib.pyplot as plt
from imageio import imread
import multiprocessing as mp
from glob import iglob

In [ ]:
bson_file = '/kaggle/input/cdiscount-image-classification-challenge/train.bson'
input_dir = os.path.abspath(os.path.join(os.getcwd(), ''))
base_dir = os.path.join(os.getcwd())
print(base_dir)
images_dir = os.path.join(base_dir, 'images')
bson_file = os.path.join(input_dir, bson_file)

In [ ]:
NCORE =  16
max_images = 2000

prod_to_category = mp.Manager().dict() # note the difference

def process(q, iolock):
    while True:
        d = q.get()
        if d is None:
            break
        product_id = str(d['_id'])
        category_id = str(d['category_id'])
        category_dir = os.path.join(images_dir, category_id)
        if not os.path.exists(category_dir):
            #category_count += 1
            try:
                os.makedirs(category_dir)
            except:
                pass

        prod_to_category[product_id] = category_id
        for e, pic in enumerate(d['imgs']):
            picture = imread(io.BytesIO(pic['picture']))
            picture_file = os.path.join(category_dir, product_id + '_' + str(e) + '.jpg')

            if not os.path.isfile(picture_file):
                #cv2.imwrite(picture_file,picture)

                plt.imsave(picture_file, picture)
            
    
q = mp.Queue(maxsize=NCORE)
iolock = mp.Lock()
pool = mp.Pool(NCORE, initializer=process, initargs=(q, iolock))

# process the file

data = bson.decode_file_iter(open(bson_file, 'rb'))
for c, d in enumerate(data):
    if (c+1) >max_images:
        break
    q.put(d)  # blocks until q below its max size

    # tell workers we're done

for _ in range(NCORE):  
    q.put(None)
pool.close()
pool.join()

print('Images saved at %s' % images_dir)

# convert back to normal dictionary
prod_to_category = dict(prod_to_category)

In [ ]:
prod_to_category = pd.DataFrame.from_dict(prod_to_category, orient='index')
prod_to_category.index.name = '_id'
prod_to_category.rename(columns={0: 'category_id'}, inplace=True)

In [ ]:
prod_to_category.head()

prod_to_category.to_csv("categories.csv")